In [2]:
%%capture
%pip install unsloth

In [3]:
from transformers import TextStreamer
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"
)

<ipython-input-3-877a0d942937>:2: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [4]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

In [5]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
    [
        prompt_style.format(
            "You are a professional doctor",
            "What is (are) Progressive Supranuclear Palsy ?",
            "",
        )
    ],
    return_tensors="pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a professional doctor

### Input:
What is (are) Progressive Supranuclear Palsy?

### Response:
Progressive supranuclear palsy (PSP) is a degenerative brain disorder that causes loss of control of physical movements such as walking, balance, and eye movements. It is a type of movement disorder, which is a group of conditions that affect a person's ability to control their bodily movements. PSP is also known as Steele–Richardson–Olszewski syndrome, after the physicians who first described the condition in 1964. PSP is a rare disease, affecting about 20,000 people in the United States.<|end_of_text|>


In [9]:
from google.colab import userdata
from huggingface_hub import login
hf_token = userdata.get('HF_TOKEN')
login(hf_token)

In [10]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024 ## souvik increase it ##
dtype = None

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype
)

==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [11]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an expert medical doctor capable of accurately answering any patient's question. Please provide a detailed and compassionate response to the following medical query.

### Input:
{}

### Response:
{}"""

In [12]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["Question"]
    outputs      = examples["Answer"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = prompt_style.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [14]:
from datasets import load_dataset

dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset", split="train[0:2500]", trust_remote_code=True)
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)
dataset["text"][0]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou are an expert medical doctor capable of accurately answering any patient's question. Please provide a detailed and compassionate response to the following medical query.\n\n### Input:\nWho is at risk for Lymphocytic Choriomeningitis (LCM)? ?\n\n### Response:\nLCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.<|end_of_text|>"

In [16]:
# Load validation data (could be different slice or a real validation set)
val_dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset", split="train[2500:2800]", trust_remote_code=True)

# Apply the same formatting function
val_dataset = val_dataset.map(
    formatting_prompts_func,
    batched=True,
)


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [15]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",

    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.4.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        eval_strategy = "steps",
        eval_steps = 5,
        save_strategy = "steps",
        save_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2500 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/300 [00:00<?, ? examples/s]

In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: iamsouvik (iamsouvik-indian-statistical-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
5,1.981100,1.099937
10,1.118800,0.798261
15,1.234500,0.758937
20,1.218200,0.739459
25,1.497100,0.731051
30,0.913400,0.714262
35,1.014300,0.703883
40,1.245400,0.697019
45,1.158800,0.694851
50,1.228400,0.694910


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [31]:
!pip install wandb -q
import wandb

wandb.finish()

eval/loss,█▃▂▂▂▁▁▁▁▁▁▁
eval/runtime,█▂▂▂▁▂▁▁▁▁▂▂
eval/samples_per_second,▁▇▇▇█▇████▇▇
eval/steps_per_second,▁▇▇▇▇▇▇▇██▇▇
train/epoch,▁▁▁▂▂▂▂▂▂▂▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
train/grad_norm,▂▂▂▂▂▂▃▃█▂▁▂▁▁▂▃▁▁▁▂▁▁▂▂▂▂▂▂▂▁▁▂▁▂▁▁▂▂▁▁
train/learning_rate,▁▂███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁
train/loss,██▆▆▇▄▄▂▃▃▃▂▃▃▂▃▄▂▂▂▂▃▃▂▂▂▃▂▃▂▁▂▂▂▃▁▃▂▂▁
eval/loss,0.69366
eval/runtime,273.1335


In [19]:
new_model_online = "iamsouvik/Llama-3.1-8B-Medquad-main"
new_model_local = "Llama-3.1-8B-Medquad-main"
model.save_pretrained(new_model_local)
tokenizer.save_pretrained(new_model_local)

('Llama-3.1-8B-Medquad-main/tokenizer_config.json',
 'Llama-3.1-8B-Medquad-main/special_tokens_map.json',
 'Llama-3.1-8B-Medquad-main/tokenizer.json')

In [20]:
model.push_to_hub(new_model_online) # Online saving model on hugging face
tokenizer.push_to_hub(new_model_online) # Online saving model on hugging face

README.md:   0%|          | 0.00/590 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/iamsouvik/Llama-3.1-8B-Medquad-main


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [23]:
from IPython.display import display, Markdown

FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        prompt_style.format(
            "What is (are) Acoustic Neuroma ?",
            "",
        )
    ],
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=250,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
Markdown(response[0].split("\n\n### Response:")[1])


An acoustic neuroma is a benign (noncancerous) tumor that develops on the main nerve that runs from the brain to the inner ear. This nerve is called the vestibular cochlear nerve, and it is responsible for hearing and balance.<|end_of_text|>

In [27]:
from IPython.display import display, Markdown

FastLanguageModel.for_inference(model)

inputs = tokenizer(
    [
        prompt_style.format(
            "What is (are) Acoustic Neuroma ?",
            "",
        )
    ],
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=512,   # We can control the lenth of the answer using the token size :/
    use_cache=True,
    eos_token_id=tokenizer.eos_token_id,  # Added so that it will not end the answer abruptly. like previous cases :)
)

response = tokenizer.batch_decode(outputs)
display(Markdown(response[0].split("\n\n### Response:")[1]))



Acoustic neuroma is a benign (noncancerous) tumor that develops on the main sensory nerve that runs from the inner ear to the brain. This nerve is called the vestibulocochlear nerve or cranial nerve VIII. The vestibulocochlear nerve transmits sound signals to the brain and controls balance. The tumor is also called vestibular schwannoma or acoustic neurinoma.
                
Acoustic neuroma is usually slow-growing and may not cause symptoms for many years. If symptoms do occur, they may include hearing loss, dizziness, ringing in the ear, and facial numbness. In some cases, symptoms occur only on one side of the body.
                
Acoustic neuroma may be found during routine hearing tests or imaging tests for other conditions. If the tumor is small, the doctor may recommend regular follow-up exams and observation. If the tumor is larger, the doctor may recommend surgery to remove the tumor or radiation therapy to shrink the tumor.<|end_of_text|>

In [30]:
from IPython.display import display, Markdown

FastLanguageModel.for_inference(model)

inputs = tokenizer(
    [
        prompt_style.format(
            "What is (are) Malabsorption Syndromes ?",
            "",
        )
    ],
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=512,   # Increased
    use_cache=True,
    eos_token_id=tokenizer.eos_token_id,  # Added EOS handling
)

response = tokenizer.batch_decode(outputs)
display(Markdown(response[0].split("\n\n### Response:")[1]))



Malabsorption syndromes are disorders in which the body has trouble absorbing nutrients from food. Symptoms can include diarrhea, bloating, gas, abdominal pain, and weight loss. The most common malabsorption syndromes are celiac disease, lactose intolerance, and inflammatory bowel disease.<|end_of_text|>

In [28]:
from IPython.display import display, Markdown

FastLanguageModel.for_inference(model)

inputs = tokenizer(
    [
        prompt_style.format(
            "How to diagnose Plasma Cell Neoplasms (Including Multiple Myeloma) ?",
            "",
        )
    ],
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=512,   # Increased
    use_cache=True,
    eos_token_id=tokenizer.eos_token_id,  # Added EOS handling
)

response = tokenizer.batch_decode(outputs)
display(Markdown(response[0].split("\n\n### Response:")[1]))


The diagnosis of plasma cell neoplasms is made by a combination of laboratory tests and physical examinations.  The diagnosis is based on a complete medical history and physical examination, blood and urine tests, X-rays, and a bone marrow test.  A bone marrow test is the most important test for diagnosing plasma cell neoplasms.  The test is performed by removing a sample of bone marrow from the hipbone with a long, thin needle.  The bone marrow sample is sent to a laboratory where it is examined under a microscope to look for abnormal plasma cells.  Other tests, including a biopsy of the bone, may be done to help diagnose and stage the disease.  A biopsy is the removal of a small piece of tissue for examination under a microscope.  A bone marrow transplant may be recommended for some patients.  A bone marrow transplant is a procedure in which the bone marrow is replaced by highly toxic drugs or radiation therapy, or by bone marrow from a donor.<|end_of_text|>